# Generator of SQL samples
Samples that are generated by this generator could be more sophisticated and more true to SQL, so for example samples that contain multiple nested queries in IN and EXISTS blocks in WHERE statement but it would be just arts for the sake of arts.

This samples have to only show that model can learn on them little more complex structure than just simple mathematic expressions

# Example
SELECT COLUMN_ID, COLUMN_ID, COLUMN_ID FROM TABLE_NAME WHERE COLUMN_ID IN ( STRING, STRING ) OR COLUMN_ID ( NUM, NUM )

WHERE COLUMN_ID FROM TABLE_NAME EXISTS ( subquery )

WHERE COLUMN_ID FROM TABLE_NAME LIKE STRING
WHERE COLUMN_ID FROM TABLE_NAME 
WHERE COLUMN_ID FROM TABLE_NAME LIMIT NUM

In [ ]:
import numpy as np
import random
from sql_simple_samples_generator import sql_simple_decode_sample, sql_simple_decode_sample_into_text

In [ ]:
cols = ['COL_ZERO', 'COL_TWO', 'COL_THREE', 'COL_FOUR', 'COL_FIVE', 'COL_SIX', 'COL_SEVEN']
frm = 'FROM'

literals = ['STRING', 'NUM']
concats = ['AND', 'OR']

sql_s_vocabulary = ['SELECT'] + cols + [',',
    'FROM',
    'TABLE_NAME',
    'WHERE',
    'IN',
    'EXISTS',
    # 'JOIN', # joins introduce a lot of complexity, will try without it and if it goes well add it later
    'AND',
    'OR',
    'LIKE',
    '(',
    'STRING',
    'NUM',
    ')',
    'LIMIT',
    'EMPTY'
  ]

sqlSDict = {el:idx for idx,el in enumerate(sql_s_vocabulary)}
len(sql_s_vocabulary)

In [ ]:
global_recursive_counter = 0

def select_col_not_in(in_cols):
  cc = [it for it in cols if it not in in_cols]
  return random.choice(cc)

def gen_core():
  q = []
  q.append('SELECT')
  c = cols[random.randint(0, len(cols)-1)]
  output_cols = [c]
  q.append(c)
  temp_cols = [it for it in cols if it not in output_cols]
  for cc in random.sample(temp_cols, k=random.randint(0, len(temp_cols)-1)):
    q.append(',')
    q.append(cc)
    output_cols.append(cc)
  q.append(frm)
  q.append('TABLE_NAME')
  return q, output_cols

def gen_IN_conditions(in_cols):
  if random.random() < 0.5:
    val_type = 'STRING'
  else:
    val_type = 'NUM'
  q = []
  q.append(random.choice(in_cols))
  q.append('IN')
  q.append('(')
  q.append(val_type)
  for _ in range(random.randint(0, 16)):
    q.append(',')
    q.append(val_type)
  q.append(')')
  return q


def gen_EXISTS_conditions(in_cols):
  global global_recursive_counter
  global_recursive_counter += 1
  if global_recursive_counter > 10:
    return gen_LIKE_conditions(in_cols)
  q = []
  q.append('EXISTS')
  q.append('(')
  q.append('SELECT')
  q.append(random.choice(in_cols))
  q.append(frm)
  q.append('TABLE_NAME')
  q.append('WHERE')
  q += gen_condition(in_cols)
  q.append(')')
  global_recursive_counter = 0
  return q


def gen_LIKE_conditions(in_cols):
  q = []
  q.append(random.choice(in_cols))
  q.append('LIKE')
  q.append('STRING')
  return q


def gen_condition(in_cols):
  possible_conditions = [[0.3, gen_IN_conditions], [0.6, gen_LIKE_conditions], [1.0, gen_EXISTS_conditions]]
  rnJesus = random.random()
  for condition in possible_conditions:
    if rnJesus < condition[0]:
      return condition[1](in_cols)


def gen_conditions(in_cols):
  q = []
  if random.random() < 0.2:
    return q

  q.append('WHERE')
  q += gen_condition(in_cols)
  return q


def sql_simple_gen_sample(max_len_of_samples=None):
  q, used_cols = gen_core()
  q += gen_conditions(used_cols)
  query = [sqlSDict[it] for it in q]
  if max_len_of_samples is not None:
    empty_values = [sqlSDict['EMPTY'] for _ in range(max_len_of_samples - len(query))]
    query += empty_values

  return query


def sql_simple_generate_samples(num_of_samples, max_len_of_samples):
  return np.asarray([sql_simple_gen_sample(max_len_of_samples) for _ in range(num_of_samples)])


def check_max_length():
  lens = [len(sql_simple_gen_sample()) for _ in range(10_000_000)]
  return max(lens)

In [ ]:
print(check_max_length()) # 128, 132, 132, 132

In [ ]:
sample = sql_simple_gen_sample()
print(len(sample))
print(sample)
decoded = sql_simple_decode_sample(sql_s_vocabulary, sample)
print(decoded)
print(sql_simple_decode_sample_into_text(sql_s_vocabulary, sample))

In [ ]:
num_of_samples = 32_000
len_of_samples = 140

samples = sql_simple_generate_samples(num_of_samples, len_of_samples)
samples.shape

In [ ]:
data_dir_path = "..\\data\\sql_simple"
for idx, sample in enumerate(samples):
  np.savetxt(f"{data_dir_path}\\sample_no{idx}.txt", sample, newline=" ", fmt="%u")